In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer, MQRNNEstimator, TemporalFusionTransformerEstimator
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/pollution.csv')

In [4]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'date'], infer_datetime_format=True)

In [6]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0,2010-01-02 00:00:00
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0,2010-01-02 01:00:00
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0,2010-01-02 02:00:00
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0,2010-01-02 03:00:00
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [7]:
df.columns

Index(['date', 'pollution', 'dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain',
       'NE', 'NW', 'SE', 'cv', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(35040, 13)

In [10]:
test_df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
35040,2014-01-01 00:00:00,24.0,-20,7.0,1014.0,143.48,0,0,0,1,0,0,2014-01-01 00:00:00
35041,2014-01-01 01:00:00,53.0,-20,7.0,1013.0,147.50,0,0,0,1,0,0,2014-01-01 01:00:00
35042,2014-01-01 02:00:00,65.0,-20,6.0,1013.0,151.52,0,0,0,1,0,0,2014-01-01 02:00:00
35043,2014-01-01 03:00:00,70.0,-20,6.0,1013.0,153.31,0,0,0,1,0,0,2014-01-01 03:00:00
35044,2014-01-01 04:00:00,79.0,-18,3.0,1012.0,0.89,0,0,0,0,0,1,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

#### training

In [13]:
K = 12
n_hours = 24*3

In [14]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=200,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-06,)

estimator = TemporalFusionTransformerEstimator(
    prediction_length=K,
    freq="H",
#     use_feat_dynamic_real=True,
    batch_size=32,
    trainer=trainer,
    dropout_rate=0.3,
    hidden_dim=64,
    num_heads=2,
#     time_features=['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv'],
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:28<00:00,  1.13it/s, epoch=77/200, avg_epoch_loss=8.27]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:32<00:00,  1.08it/s, epoch=153/200, avg_epoch_loss=8.33]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:33<00:00,  1.07it/s, epoch=200/200, avg_epoch_loss=7.98]


#### predicted y

In [15]:
quantiles = ['0.5', '0.1', '0.9']
q50 = 0

In [16]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].forecast_array
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135]
(12,)
------
1
[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135  72.70623779  64.56420135  57.2887764
  51.17324448  46.38433838  42.61324692  39.69888687  37.60065079
  36.31028748  35.81042862  36.0607872   36.9483757 ]
(24,)
------
2
[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135  72.70623779  64.56420135  57.2887764
  51.17324448  46.38433838  42.61324692  39.69888687  37.60065079
  36.31028748  35.81042862  36.0607872   36.9483757   78.58359528
  69.80039215  62.62897873  56.23416519  50.96475601  46.63504028
  43.26541138  40.84905243  39.36236954  38.82948685  39.23214722
  40.61756516]
(36,)
------
3


[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135  72.70623779  64.56420135  57.2887764
  51.17324448  46.38433838  42.61324692  39.69888687  37.60065079
  36.31028748  35.81042862  36.0607872   36.9483757   78.58359528
  69.80039215  62.62897873  56.23416519  50.96475601  46.63504028
  43.26541138  40.84905243  39.36236954  38.82948685  39.23214722
  40.61756516  84.8937149   76.55504608  69.39595795  62.88331985
  57.19949341  52.41381836  48.75899124  46.2253685   44.75679398
  44.27138901  44.61713791  46.50987625  70.69350433  65.81154633
  60.07644272  53.12099838  46.45785904  41.05132675  37.04019547
  34.20416641  32.45991516  31.86438179  33.33102798  38.82216263
  72.05731201  69.82623291  65.73729706  60.29752731  54.73970032
  50.16113663  46.89621353  44.82971191  43.96830368  45.11046219
  50.09550095  57.9629097   72.81350708  72.74802399  70.85637665
  67.749343

[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135  72.70623779  64.56420135  57.2887764
  51.17324448  46.38433838  42.61324692  39.69888687  37.60065079
  36.31028748  35.81042862  36.0607872   36.9483757   78.58359528
  69.80039215  62.62897873  56.23416519  50.96475601  46.63504028
  43.26541138  40.84905243  39.36236954  38.82948685  39.23214722
  40.61756516  84.8937149   76.55504608  69.39595795  62.88331985
  57.19949341  52.41381836  48.75899124  46.2253685   44.75679398
  44.27138901  44.61713791  46.50987625  70.69350433  65.81154633
  60.07644272  53.12099838  46.45785904  41.05132675  37.04019547
  34.20416641  32.45991516  31.86438179  33.33102798  38.82216263
  72.05731201  69.82623291  65.73729706  60.29752731  54.73970032
  50.16113663  46.89621353  44.82971191  43.96830368  45.11046219
  50.09550095  57.9629097   72.81350708  72.74802399  70.85637665
  67.749343

[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135  72.70623779  64.56420135  57.2887764
  51.17324448  46.38433838  42.61324692  39.69888687  37.60065079
  36.31028748  35.81042862  36.0607872   36.9483757   78.58359528
  69.80039215  62.62897873  56.23416519  50.96475601  46.63504028
  43.26541138  40.84905243  39.36236954  38.82948685  39.23214722
  40.61756516  84.8937149   76.55504608  69.39595795  62.88331985
  57.19949341  52.41381836  48.75899124  46.2253685   44.75679398
  44.27138901  44.61713791  46.50987625  70.69350433  65.81154633
  60.07644272  53.12099838  46.45785904  41.05132675  37.04019547
  34.20416641  32.45991516  31.86438179  33.33102798  38.82216263
  72.05731201  69.82623291  65.73729706  60.29752731  54.73970032
  50.16113663  46.89621353  44.82971191  43.96830368  45.11046219
  50.09550095  57.9629097   72.81350708  72.74802399  70.85637665
  67.749343

[100.39278412  87.01515961  73.57805634  61.35212326  52.16782761
  45.57029724  40.74831009  37.33299637  34.98189163  33.42738342
  32.54925919  32.29629135  72.70623779  64.56420135  57.2887764
  51.17324448  46.38433838  42.61324692  39.69888687  37.60065079
  36.31028748  35.81042862  36.0607872   36.9483757   78.58359528
  69.80039215  62.62897873  56.23416519  50.96475601  46.63504028
  43.26541138  40.84905243  39.36236954  38.82948685  39.23214722
  40.61756516  84.8937149   76.55504608  69.39595795  62.88331985
  57.19949341  52.41381836  48.75899124  46.2253685   44.75679398
  44.27138901  44.61713791  46.50987625  70.69350433  65.81154633
  60.07644272  53.12099838  46.45785904  41.05132675  37.04019547
  34.20416641  32.45991516  31.86438179  33.33102798  38.82216263
  72.05731201  69.82623291  65.73729706  60.29752731  54.73970032
  50.16113663  46.89621353  44.82971191  43.96830368  45.11046219
  50.09550095  57.9629097   72.81350708  72.74802399  70.85637665
  67.749343

...

[100.39278412  87.01515961  73.57805634 ...  16.85168266  18.61924553
  19.0013752 ]
(99780,)
------
8315
[100.39278412  87.01515961  73.57805634 ...  22.91542625  23.07460022
  23.18548393]
(99792,)
------
8316
[100.39278412  87.01515961  73.57805634 ...  22.17904472  22.21814156
  22.32286072]
(99804,)
------
8317
[100.39278412  87.01515961  73.57805634 ...  22.35865211  22.3917141
  22.24246216]
(99816,)
------
8318
[100.39278412  87.01515961  73.57805634 ...  35.94405746  35.96381378
  35.85850906]
(99828,)
------
8319
[100.39278412  87.01515961  73.57805634 ...  41.7190094   41.37155533
  40.48077011]
(99840,)
------
8320
[100.39278412  87.01515961  73.57805634 ...  39.12569046  38.16875076
  37.1220665 ]
(99852,)
------
8321
[100.39278412  87.01515961  73.57805634 ...  35.49264908  35.02331543
  33.7513237 ]
(99864,)
------
8322
[100.39278412  87.01515961  73.57805634 ...  37.96723557  36.79125214
  35.2617569 ]
(99876,)
------
8323
[100.39278412  87.01515961  73.57805634 ...  34

[100.39278412  87.01515961  73.57805634 ...  15.17668247  14.29636383
  13.32853317]
(100728,)
------
8394
[100.39278412  87.01515961  73.57805634 ...  16.11489677  13.96245098
  11.73798275]
(100740,)
------
8395
[100.39278412  87.01515961  73.57805634 ...  42.96669006  40.62966156
  37.74985886]
(100752,)
------
8396
[100.39278412  87.01515961  73.57805634 ...  18.30791664  16.35506439
  14.6450243 ]
(100764,)
------
8397
[100.39278412  87.01515961  73.57805634 ...  15.39077759  14.06341839
  12.80597687]
(100776,)
------
8398
[100.39278412  87.01515961  73.57805634 ...   8.43399143   7.5917325
   6.69581556]
(100788,)
------
8399
[100.39278412  87.01515961  73.57805634 ...   7.35142279   6.54702902
   5.71239328]
(100800,)
------
8400
[100.39278412  87.01515961  73.57805634 ...   4.72392082   4.2354064
   3.87396812]
(100812,)
------
8401
[100.39278412  87.01515961  73.57805634 ...   5.87387657   5.51928473
   5.30343151]
(100824,)
------
8402
[100.39278412  87.01515961  73.57805634

[100.39278412  87.01515961  73.57805634 ... 118.4595871  113.33129883
 107.04148102]
(101676,)
------
8473
[100.39278412  87.01515961  73.57805634 ... 108.4539566  102.86141205
  98.47937012]
(101688,)
------
8474
[100.39278412  87.01515961  73.57805634 ... 126.39235687 120.42901611
 116.77204132]
(101700,)
------
8475
[100.39278412  87.01515961  73.57805634 ...  79.19322205  79.73253632
  81.24698639]
(101712,)
------
8476
[100.39278412  87.01515961  73.57805634 ...  74.93533325  74.65365601
  76.7980423 ]
(101724,)
------
8477
[100.39278412  87.01515961  73.57805634 ...  73.96127319  77.90921783
  87.26638794]
(101736,)
------
8478
[100.39278412  87.01515961  73.57805634 ...  64.55480194  73.17434692
  89.8236084 ]
(101748,)
------
8479
[100.39278412  87.01515961  73.57805634 ...  48.22672653  57.76446533
  78.15724945]
(101760,)
------
8480
[100.39278412  87.01515961  73.57805634 ...  82.00997162 104.02099609
 126.00319672]
(101772,)
------
8481
[100.39278412  87.01515961  73.578056

[100.39278412  87.01515961  73.57805634 ... 135.52830505 147.89099121
 156.15524292]
(102636,)
------
8553
[100.39278412  87.01515961  73.57805634 ... 150.32807922 160.19136047
 167.18920898]
(102648,)
------
8554
[100.39278412  87.01515961  73.57805634 ... 166.69075012 175.63293457
 183.1373291 ]
(102660,)
------
8555
[100.39278412  87.01515961  73.57805634 ... 205.30247498 215.62931824
 225.07966614]
(102672,)
------
8556
[100.39278412  87.01515961  73.57805634 ... 197.01017761 204.1476593
 210.70703125]
(102684,)
------
8557
[100.39278412  87.01515961  73.57805634 ... 231.39176941 236.39347839
 242.17810059]
(102696,)
------
8558
[100.39278412  87.01515961  73.57805634 ... 276.73321533 280.28530884
 279.61380005]
(102708,)
------
8559
[100.39278412  87.01515961  73.57805634 ... 235.8105011  228.83863831
 219.35017395]
(102720,)
------
8560
[100.39278412  87.01515961  73.57805634 ... 210.59631348 201.77241516
 190.38616943]
(102732,)
------
8561
[100.39278412  87.01515961  73.5780563

[100.39278412  87.01515961  73.57805634 ... 229.51664734 234.4152832
 262.01416016]
(103560,)
------
8630
[100.39278412  87.01515961  73.57805634 ... 256.48031616 289.39395142
 318.57861328]
(103572,)
------
8631
[100.39278412  87.01515961  73.57805634 ... 266.11349487 292.63961792
 306.12631226]
(103584,)
------
8632
[100.39278412  87.01515961  73.57805634 ... 131.88922119 132.47753906
 124.45597076]
(103596,)
------
8633
[100.39278412  87.01515961  73.57805634 ...  57.62659836  47.80931091
  37.41550064]
(103608,)
------
8634
[100.39278412  87.01515961  73.57805634 ...  53.10967636  44.71878052
  37.27626801]
(103620,)
------
8635
[100.39278412  87.01515961  73.57805634 ... 195.03540039 187.4650116
 180.79631042]
(103632,)
------
8636
[100.39278412  87.01515961  73.57805634 ... 151.42027283 146.01278687
 140.21356201]
(103644,)
------
8637
[100.39278412  87.01515961  73.57805634 ... 137.40594482 132.47389221
 126.71547699]
(103656,)
------
8638
[100.39278412  87.01515961  73.57805634

In [17]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(8677, 12)

In [18]:
test_ds_predict_q50_supervised[0:3]

array([[100.39278412,  87.01515961,  73.57805634,  61.35212326,
         52.16782761,  45.57029724,  40.74831009,  37.33299637,
         34.98189163,  33.42738342,  32.54925919,  32.29629135],
       [ 72.70623779,  64.56420135,  57.2887764 ,  51.17324448,
         46.38433838,  42.61324692,  39.69888687,  37.60065079,
         36.31028748,  35.81042862,  36.0607872 ,  36.9483757 ],
       [ 78.58359528,  69.80039215,  62.62897873,  56.23416519,
         50.96475601,  46.63504028,  43.26541138,  40.84905243,
         39.36236954,  38.82948685,  39.23214722,  40.61756516]])

#### actual y

In [19]:
from helper import series_to_supervised, stage_series_to_supervised

In [20]:
test_df_supervised = series_to_supervised(test_df[['pollution']], n_hours, K)

In [21]:
test_df_supervised.iloc[:, -K:].shape

(8677, 12)

In [22]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [23]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),var1(t+10),var1(t+11)
35112,85.0,86.0,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0
35113,86.0,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0
35114,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0
35115,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0,176.0
35116,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0,176.0,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43784,8.0,8.0,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0
43785,8.0,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0
43786,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0,10.0
43787,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0,10.0,8.0


#### performance on test set

In [24]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 36.323
RMSE = 60.134
